<a href="https://colab.research.google.com/github/dataforgoodfr/batch8_mednum/blob/master/notebooks/acces_information.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## 1. Liste des médiathèques


In [ ]:
# get mediatheques map

In [ ]:
from google.colab import drive
drive.mount("/content/drive", force_remount=True)

Mounted at /content/drive


In [ ]:
ls drive/'My Drive'/'Colab Notebooks'/

carte-des-bibliotheques.csv  correspondances-code-insee-code-postal.csv
carte_mediatheque.ipynb


In [ ]:
cd drive/'My Drive'/'Colab Notebooks'/

/content/drive/My Drive/Colab Notebooks


In [ ]:
import pandas as pd

df = pd.read_csv('carte-des-bibliotheques.csv', header=0, sep=';')
print(df.shape)
df.head()

(844, 21)


,Nom structure,Adresse,Code postal,Département,Ville,Année(s) subvention Région,Montant subvention 2000,Montant subvention 2001,Montant subvention 2002,Montant subvention 2003,Montant subvention 2004,Montant subvention 2005,Montant subvention 2006,Montant subvention 2007,Montant subvention 2008,Montant subvention 2009,Montant subvention 2010,Montant subvention 2011,Montant subvention 2012,Montant subvention 2013,wgs84
0,MÉDIATHÈQUE SAINT-JOHN PERSE,2 rue Édouard Poisson,93300,93,AUBERVILLIERS,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"48.9104747,2.385008"
1,MÉDIATHÈQUE GEORGES PÉREC,8-10 rue du Marqis de Raies,91080,91,COURCOURONNES,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"48.6293899,2.4135832"
2,MÉDIATHÈQUE LES MUSES,101 rue de la Division Leclerc,91310,91,LINAS,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"48.6305113,2.2650636"
3,MÉDIATHÈQUE MUNICIPALE,2 route de la Creusée,91530,91,LE-VAL-SAINT-GERMAIN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"48.5657217,2.0601725"
4,BIBLIOTHÈQUE LE GRENIER,5 place des Droits de l'Homme,91800,91,BOUSSY-SAINT-ANTOINE,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"48.6891076,2.5297452"


In [ ]:
df = df[['Code postal', 'Ville', 'Département']]
df['Code postal'] = df['Code postal'].astype(str)
df.head()

,Code postal,Ville,Département
0,93300,AUBERVILLIERS,93
1,91080,COURCOURONNES,91
2,91310,LINAS,91
3,91530,LE-VAL-SAINT-GERMAIN,91
4,91800,BOUSSY-SAINT-ANTOINE,91


In [ ]:
# mapping code postal - code INSEE
mapping = pd.read_csv('correspondances-code-insee-code-postal.csv', header=0, sep=';')
mapping.rename(columns={'Code Postal': 'Code postal'}, inplace=True)
mapping.head()

,Code INSEE,Code postal,Commune,Département,Région,Statut,Altitude Moyenne,Superficie,Population,geo_point_2d,geo_shape,ID Geofla,Code Commune,Code Canton,Code Arrondissement,Code Département,Code Région
0,91645,91370,VERRIERES-LE-BUISSON,ESSONNE,ILE-DE-FRANCE,Commune simple,121.0,999.0,15.5,"48.75044312,2.25171297214","{""type"": ""Polygon"", ""coordinates"": [[[2.238024...",16275,645,3,3,91,11
1,77133,77126,COURCELLES-EN-BASSEE,SEINE-ET-MARNE,ILE-DE-FRANCE,Commune simple,88.0,1082.0,0.2,"48.4125606521,3.05294050556","{""type"": ""Polygon"", ""coordinates"": [[[3.076046...",31428,133,20,3,77,11
2,91378,91730,MAUCHAMPS,ESSONNE,ILE-DE-FRANCE,Commune simple,150.0,313.0,0.3,"48.5272680908,2.19718165044","{""type"": ""Polygon"", ""coordinates"": [[[2.203466...",30975,378,9,1,91,11
3,77243,77400,LAGNY-SUR-MARNE,SEINE-ET-MARNE,ILE-DE-FRANCE,Chef-lieu canton,71.0,579.0,20.2,"48.8730701858,2.70978081313","{""type"": ""Polygon"", ""coordinates"": [[[2.727542...",17000,243,14,5,77,11
4,94003,94110,ARCUEIL,VAL-DE-MARNE,ILE-DE-FRANCE,Chef-lieu canton,70.0,232.0,19.5,"48.8058803597,2.33351024984","{""type"": ""Polygon"", ""coordinates"": [[[2.343851...",32123,3,34,3,94,11


In [ ]:
mapping.drop_duplicates(inplace=True)

In [ ]:
df.merge(mapping[['Code INSEE', 'Code postal']], how='inner', on='Code postal').drop_duplicates()  

,Code postal,Ville,Département,Code INSEE
0,93300,AUBERVILLIERS,93,93001
4,91080,COURCOURONNES,91,91182
5,91310,LINAS,91,91333
6,91310,LINAS,91,91425
7,91310,LINAS,91,91339
...,...,...,...,...
2722,77440,ARMENTIÈRES-EN-BRIE,77,77490
2723,77440,ARMENTIÈRES-EN-BRIE,77,77231
2724,77440,ARMENTIÈRES-EN-BRIE,77,77235
2725,77200,TORCY,77,77468


In [ ]:
df.groupby('Code postal').count().sort_values(by='Ville')[::-1]

,Ville,Département
Code postal,,
78000,9,9
77400,7,7
77130,7,7
78490,6,6
95000,6,6
...,...,...
92104,1,1
92110,1,1
92120,1,1


## 2. Liste des centres sociaux

In [ ]:
import json
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
from urllib import request

import time


def timeit(method):
    def timed(*args, **kw):
        ts = time.time()
        result = method(*args, **kw)
        te = time.time()        
        if 'log_time' in kw:
            name = kw.get('log_name', method.__name__.upper())
            kw['log_time'][name] = int((te - ts) * 1000)
        else:
            print('%r  %2.2f ms' % \
                  (method.__name__, (te - ts) * 1000))
        return result    
    return timed

In [ ]:
@timeit
def scrap_table_page(url='https://www.senacs.fr/structure/csx?page=', page_number=1):
    """
    Generate a request on senacs page for a given 
    page number to get the name and location of 
    each social center.
    """

    url_page = url + str(page_number)
    request_text = request.urlopen(url_page).read()
    page = BeautifulSoup(request_text, 'html.parser')

    table = page.find('table', {'class': 'table table-striped table-hover'}).findAll('td')

    # fill the values of id, name and city
    ids = [id.getText() for id in table[::4]]
    names = [name.getText() for name in table[1::4]]
    cities = [city.getText() for city in table[2::4]]

    df = pd.DataFrame({'id': ids, 
                      'nom': names,
                      'ville': cities}
                      )
    df['page'] = page_number

    return df


def stack_pages(pages=range(91)):
    """
    Concatenate all empruntis scrapped data per region.
    """

    data = pd.DataFrame()    # empty Dataframe to fill
    for page_number in pages:
        temp = scrap_table_page(page_number=page_number)
        data = pd.concat([data, temp], axis=0)

    return data

In [ ]:
data = stack_pages()

In [ ]:
data['code_postal'] = data['ville'].apply(lambda x: x.split('-')[0])
data['ville'] = data['ville'].apply(lambda x: x.split('-')[1])

In [ ]:
data

,id,nom,ville,page,code_postal
0,91,CENTRE SOCIAL DES GRANDES BORNES,Goussainville,0,95190
1,92,CENTRE SOCIAL ARCHIPELIA,PARIS 20EME ARRONDISSEMENT,0,75020
2,93,Espace Socioculturel Val de Charente,Ruffec,0,16700
3,94,CENTRE SOCIAL Maison Des Habitants Champaret,Bourgoin,0,38300
4,95,CENTRE SOCIAL MAISON DE LA CHALLE,Éragny,0,95610
...,...,...,...,...,...
6,16264,CENTRE SOCIOCULTUREL MUNICIPAL DE SAINT-DIZIER,Saint,90,52100
7,16267,ESPACE DE VIE SOCIALE L.E.P.H.A.R.E,Aniche,90,59580
8,16351,ASSOCIATION FERME BEAUREPAIRE,Boulogne,90,62200
9,16355,OFFICE DE LA JEUNESSE,Bruay,90,62700


## 3. Distance des communes à l'accès le plus proche


In [ ]:
# copy geoson file with every 'Franceservices' locations

import os

if 'COLAB_GPU' in os.environ:  # this is always set on Colab, the value is 0 or 1 depending on whether a GPU is attached
    from google.colab import auth
    auth.authenticate_user()

    !git clone https://github.com/cget-carto/France-services.git
    !mv France-services/data/france_services.geojson .

Cloning into 'France-services'...
remote: Enumerating objects: 260, done.
remote: Counting objects: 100% (260/260), done.
remote: Compressing objects: 100% (169/169), done.
remote: Total 260 (delta 108), reused 221 (delta 78), pack-reused 0
Receiving objects: 100% (260/260), 2.12 MiB | 14.58 MiB/s, done.
Resolving deltas: 100% (108/108), done.


In [ ]:
ls

adc.json  France-services/  france_services.geojson  sample_data/


In [ ]:
import pandas as pd
import json

with open("france_services.geojson", "r") as read_file: 
    fs = json.load(read_file) 

In [ ]:
data = pd.DataFrame()
for n, temp in enumerate(fs['features']): 
 
    df_temp = pd.DataFrame.from_records([{'latitude': temp['properties']['LATITUDE'], 
                            'longitude': temp['properties']['LATITUDE'], 
                            'departement': temp['properties']['DEPARTEMENT'],
                            'insee_com': temp['properties']['insee_com'],
                            'code_postal': temp['properties']['code_postal'], 
                            'lib_france_services': temp['properties']['lib_france_services']
                            }])
    
    data = pd.concat([data, df_temp], axis=0)

In [ ]:
data

,latitude,longitude,departement,insee_com,code_postal,lib_france_services
0,46.105747,46.105747,1,01033,01200,Valserhône
0,44.031548,44.031548,12,12197,12170,Réquista « Le Bercail »
0,44.475185,44.475185,12,12138,12330,Conques-Marcillac
0,49.291400,49.291400,14,14514,14130,Terre d’Auge
0,49.126169,49.126169,14,14057,14370,Val ès Dunes
...,...,...,...,...,...,...
0,47.617932,47.617932,89,89368,89520,Saint-Sauveur-en-Puisaye
0,48.980007,48.980007,95,95219,95120,Ermont
0,47.181000,47.181000,25,25527,25410,Saint Vit
0,43.208769,43.208769,31,31375,31310,Volvestre – Antenne Montesquieu


In [ ]:
!pip install pgeocode

### Calculer la distance entre le centre des communes et la prefecture, sous-prefecture ou france-services la plus proche en km. 

- scrapper prefecture / sous-prefecture https://fr.wikipedia.org/wiki/Liste_des_pr%C3%A9fectures_de_France
- obtenir coordonnées des communes: OK via codes postaux + librairie `pgecode`

In [ ]:
import pgeocode
dist = pgeocode.GeoDistance('fr')

!git clone https://github.com/dataforgoodfr/batch8_mednum.git
!mv batch8_mednum/data/table_iris_insee_epci.csv .
!ls

Cloning into 'batch8_mednum'...
remote: Enumerating objects: 53, done.
remote: Counting objects: 100% (53/53), done.
remote: Compressing objects: 100% (49/49), done.
remote: Total 53 (delta 19), reused 8 (delta 0), pack-reused 0
Unpacking objects: 100% (53/53), done.
adc.json       France-services		sample_data
batch8_mednum  france_services.geojson	table_iris_insee_epci.csv


In [ ]:
#data['distance'] = dist.query_postal_code(data[''].values, data[''].values)

In [ ]:
commune = pd.read_csv('table_iris_insee_epci.csv')
commune

,Unnamed: 0,CODE_IRIS,TYP_IRIS,CODE_INSEE,LIBCOM,EPCI,NATURE_EPCI,MODIF_IRIS,GRD_QUART,UU2010,REG,DEP
0,0,010010000,Z,01001,L'Abergement-Clémenciat,200069193,CC,0,0100100,01000,84,01
1,1,010020000,Z,01002,L'Abergement-de-Varey,240100883,CC,0,0100200,01000,84,01
2,2,010040101,H,01004,Ambérieu-en-Bugey,240100883,CC,0,0100401,01302,84,01
3,3,010040102,H,01004,Ambérieu-en-Bugey,240100883,CC,0,0100401,01302,84,01
4,4,010040201,H,01004,Ambérieu-en-Bugey,240100883,CC,0,0100402,01302,84,01
...,...,...,...,...,...,...,...,...,...,...,...,...
49418,49418,976170301,H,97617,Tsingoni,200059871,CC,0,9761703,9F209,6,976
49419,49419,976170302,H,97617,Tsingoni,200059871,CC,0,9761703,9F209,6,976
49420,49420,976170303,D,97617,Tsingoni,200059871,CC,0,9761703,9F209,6,976
49421,49421,976170304,D,97617,Tsingoni,200059871,CC,0,9761703,9F209,6,976


In [ ]:
# calculer le france-service (ou prefecture, sous-prefecture etc.) le plus proche de chaque commune et bueno 

## 4. Part des moins de 25 ans

In [ ]:
from google.colab import drive
drive.mount("/content/drive", force_remount=True)

Mounted at /content/drive


In [ ]:
demo = pd.read_csv('/content/drive/My Drive/Colab Notebooks/data_DEMO_2020_V1_COM.csv', sep=';')
demo[['CODGEO', 'POP_MUN', 'TX_TOT_0A24']]

,CODGEO,POP_MUN,TX_TOT_0A24
0,01004,14359,"32,5"
1,01031,3646,"32,1"
2,01034,8909,"29,5"
3,01053,40490,"31,3"
4,01160,9236,"29,1"
...,...,...,...
786,97415,104332,"35,4"
787,97416,81415,"37,3"
788,97418,32605,"37,5"
789,97420,22209,"39,7"
